# Set Coding Environment

## Module Imports

In [1]:
import ee
from pprint import pprint

## Mount Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/LULC_Experiments_Chahat_Ananjan_Saketh

/content/drive/MyDrive/LULC_Experiments_Chahat_Ananjan_Saketh


## Authenticate to Google Earth Engine

In [3]:
ee.Authenticate() #Uncomment this whenever needed, once done usually not needed for 1-2 days

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=iSEou66l2D11V7P6fPI4LdUyMR22JmmXVwPXEeQqg7k&tc=L5tnv5eXf6fX0UNVr1lt7Rv82kuUJPBIedeK02TwegE&cc=k4mnRiLBeGRETUkJ4auWZJO8cntJzZptmm4TgxglVR4

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXmqP9ToCmj_dTCFnerTdKwiK0oeYFgxgWLZZfQ6Rt2OEuh4jz1znkU

Successfully saved authorization token.


In [4]:
ee.Initialize()

# Function Definitions

In [5]:
'''
Function definition to combine L1, L2, and L3 outputs of a given region
'''
def combine_LULC_images(L1_assetpath, L2_assetpath, L3_assetpath, roi_boundary):
  L1_output = ee.Image(L1_assetpath)
  L2_output = ee.Image(L2_assetpath)
  L3_output = ee.Image(L3_assetpath) #.select(['class','confidence_L3']).rename(['Predicted_L3_Label','confidence_L3'])

  greenery_mask_ = L1_output.select('Predicted_L1_Label').expression("b('Predicted_L1_Label') == 1").clip(roi_boundary)
  crop_mask = L2_output.select('Predicted_L2_Label').expression("b('Predicted_L2_Label') == 5").And(L1_output.select('Predicted_L1_Label').expression("b('Predicted_L1_Label') == 1")).clip(roi_boundary)
  final_classification_output = L1_output.select('Predicted_L1_Label').where(greenery_mask_, L2_output.select('Predicted_L2_Label')).where(crop_mask, L3_output.select('Predicted_L3_Label')).rename('class').clip(roi_boundary)
  final_confidence_output = L1_output.select('confidence_L1').where(greenery_mask_, L2_output.select('confidence_L2')).where(crop_mask, L3_output.select('confidence_L3')).rename('confidence').clip(roi_boundary)

  final_output = final_classification_output.addBands(final_confidence_output)
  return final_output


In [6]:
roi_name = 'Baran'
district = ee.FeatureCollection('projects/ee-indiasat/assets/india_district_boundaries').filter(ee.Filter.eq('Name', roi_name))
roi_boundary = district.geometry()

# roi_boundary = ee.FeatureCollection('users/mtpictd/agro_eco_regions').filter(ee.Filter.eq('ae_regcode',13)).geometry()
# roi_name = 'EcoRegion13'

start_years = ['2016', '2017', '2018', '2019', '2020', '2021','2022']
end_years = ['2017', '2018', '2019', '2020', '2021', '2022','2023']

L1_prefix_path = 'projects/ee-indiasat/assets/LULC_Deliverables_WithConfidence/Modified/'
L2_prefix_path = 'projects/ee-indiasat/assets/LULC_Deliverables_WithConfidence/Modified/'
# L3_prefix_path = 'projects/ee-indiasat/assets/LULC_Results_Level3_Original/'
L3_prefix_path = 'projects/ee-indiasat/assets/LULC_Deliverables_WithConfidence/Modified/'
output_prefix_path = 'projects/ee-indiasat/assets/LULC_CombinedOutputs_WithConfidence/'

# L1_assetname_list  = [roi_name + '_2016-07-01_2017-06-30_Level1_LULCmap_30m', roi_name + '_2017-07-01_2018-06-30_Level1_LULCmap_30m', roi_name + '_2018-07-01_2019-06-30_Level1_LULCmap_30m', roi_name + '_2019-07-01_2020-06-30_Level1_LULCmap_30m', roi_name + '_2020-07-01_2021-06-30_Level1_LULCmap_30m', roi_name + '_2021-07-01_2022-06-30_Level1_LULCmap_30m']
# L2_assetname_list  = [roi_name + '_2016-07-01_2017-06-30_Level2_LULCmap_30m', roi_name + '_2017-07-01_2018-06-30_Level2_LULCmap_30m', roi_name + '_2018-07-01_2019-06-30_Level2_LULCmap_30m', roi_name + '_2019-07-01_2020-06-30_Level2_LULCmap_30m', roi_name + '_2020-07-01_2021-06-30_Level2_LULCmap_30m', roi_name + '_2021-07-01_2022-06-30_Level2_LULCmap_30m']
# L3_assetname_list  = [roi_name + '_2016-07-01_2017-06-30_Level3_LULCmap_30m', roi_name + '_2017-07-01_2018-06-30_Level3_LULCmap_30m', roi_name + '_2018-07-01_2019-06-30_Level3_LULCmap_30m', roi_name + '_2019-07-01_2020-06-30_Level3_LULCmap_30m', roi_name + '_2020-07-01_2021-06-30_Level3_LULCmap_30m', roi_name + '_2021-07-01_2022-06-30_Level3_LULCmap_30m']

num_of_years =  7
start_year_suffix = '-07-01'
end_year_suffix = '-06-30'
asset_name = ''

for i in range(num_of_years):
  # print(i)
  L1_assetpath = L1_prefix_path + roi_name + '_' + start_years[i] + start_year_suffix + '_' + end_years[i] + end_year_suffix + '_Level1_LULCmap_30m'
  L2_assetpath = L2_prefix_path + roi_name + '_' + start_years[i] + start_year_suffix + '_' + end_years[i] + end_year_suffix + '_Level2_LULCmap_30m'
  L3_assetpath = L3_prefix_path + roi_name + '_' + start_years[i] + start_year_suffix + '_' + end_years[i] + end_year_suffix + '_Level3_LULCmap_30m'

  combined_output = combine_LULC_images(L1_assetpath, L2_assetpath, L3_assetpath, roi_boundary)
  output_assetName = roi_name + '_' + start_years[i] + start_year_suffix + '_' + end_years[i] + end_year_suffix + '_LULCmap_30m'

  print("Exporting ",output_assetName)

  image_export_task = ee.batch.Export.image.toAsset(
    image = combined_output,
    description = output_assetName,
    assetId = output_prefix_path + output_assetName,
    scale = 30,
    pyramidingPolicy = {'class': 'mode'},
    maxPixels = 1e13,
    crs = 'EPSG:4326'
  )

  # image_export_task = ee.batch.Export.image.toDrive(
  #     image = combined_output,
  #     description = output_assetName,
  #     folder = 'LULC_Combined_Outputs_WithConfidence',
  #     fileNamePrefix = output_assetName,
  #     region = roi_boundary,
  #     scale = 30,
  #     maxPixels = 1e13,
  #     crs = 'EPSG:4326',
  #     fileFormat = 'GeoTIFF'
  # )

  image_export_task.start()

Exporting  Baran_2016-07-01_2017-06-30_LULCmap_30m
Exporting  Baran_2017-07-01_2018-06-30_LULCmap_30m
Exporting  Baran_2018-07-01_2019-06-30_LULCmap_30m
Exporting  Baran_2019-07-01_2020-06-30_LULCmap_30m
Exporting  Baran_2020-07-01_2021-06-30_LULCmap_30m
Exporting  Baran_2021-07-01_2022-06-30_LULCmap_30m
Exporting  Baran_2022-07-01_2023-06-30_LULCmap_30m
